<img src="./detsd/hpe_logo.png" alt="HPE Logo" width="300">

# Textual Inversion (Stable Diffusion) with Determined.AI

Stable Diffusion is a deep learning, text-to-image model released in 2022. </br>It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt. </br>It was developed by the start-up Stability AI in collaboration with a number of academic researchers and non-profit organizations.

For this exercise, we will leverage stable diffusion from Huggingface here: https://huggingface.co/runwayml/stable-diffusion-v1-5. </br>
Determined.AI is an ideal platform to finetune models from a modelhub like Huggingface. In this case, we already did the training of the model to teach it what a `det-logo-demo` means.

Here is our Determined.AI logo:</br> </br>
<img src="./detsd/det_logo.png" alt="Determined Logo" width="150">

This notebook generates images from the trained textual inversion models generated with the `detsd.DetSDTextualInversionTrainer` class and saved as Determined checkpoints.AI.

## Step 1: Creating the Pipeline

Import the `DetSDTextualInversionPipeline` class from `detsd.py` (loaded via the `--context` flag above), which will be used to generate Stable Diffusion images.

In [ ]:
import torch
from detsd import DetSDTextualInversionPipeline

Instantiate the pipeline with the default arguments:

In [ ]:
detsd_pipeline = DetSDTextualInversionPipeline()

## Step 2: Load Determined Checkpoints

We can now load textual-inversion checkpoints into the model. They are assumed to have been trained with `DetSDTextualInversionTrainer`, also contained in `detsd.py`.  These Determined checkpoints can be specified by their uuid, assuming all such checkpoints exist on the master we are currently logged into.

Please go to the Determined.AI Cluster UI and find the project, experiment and the best checkpoint for a detsd finetune experiment. Fill in the `uuids` list below with the `uuid` `str` values of the best Determined.AI checkpoint you can find. (Replace `FIX ME` with checkpoint UUID)

In [ ]:
uuids = ["FIX ME"]
detsd_pipeline.load_from_uuids(uuids)

A sample embedding is also included in this environment (with corresponding concept token `det-logo-demo`) and can be loaded in as follows:

In [ ]:
from os.path import exists
demo_concept_path = 'learned_embeddings_dict_demo.pt'
if exists(demo_concept_path):
    detsd_pipeline.load_from_checkpoint_dir(checkpoint_dir='.', learned_embeddings_filename='learned_embeddings_dict_demo.pt')

## Step 3: Generate Images

Finally, let's generate some art.

Grab the first concept which was loaded into the pipeline and store it as `first_concept`.  If no concepts were loaded above, fall back to using `brain logo, sharp lines, connected circles, concept art` as a default value for `first_concept`; vanilla Stable Diffusion is being used in this case.

In [ ]:
all_added_concepts = detsd_pipeline.all_added_concepts
if all_added_concepts:
    first_concept = all_added_concepts[0]
else:
    first_concept = 'brain logo, sharp lines, connected circles, concept art'
print(f'Using "{first_concept}" as first_concept in the below\n')
print(f'All available concepts: {all_added_concepts}')

Create a directory for saved images and an index for tracking the number of images created.

In [ ]:
save_dir = 'generated_images'
! mkdir {save_dir}
num_generated_images = 0

### Step 3.1: Running the model with a custom text prompt

The below code uses creates `batch_size * num_images_per_prompt` total images from the prompt.

If you are generating using the demo embedding with `det-logo-demo` as `first_concept`, we recommend setting the guidance scale to a relatively low value, e.g. ~3.

In [ ]:
prompt = f'a watercolor painting on textured paper of a {first_concept} using soft strokes, pastel colors, incredible composition, masterpiece'
batch_size = 2
num_images_per_prompt = 2

generator = torch.Generator(device='cuda').manual_seed(2147483647)
output = detsd_pipeline(prompt=[prompt] * batch_size,
                        num_images_per_prompt=num_images_per_prompt,
                        num_inference_steps=50,
                        generator=generator,
                        guidance_scale=7.5
                       )

Visualize and save:

In [ ]:
from pathlib import Path

for img, nsfw in zip(output.images, output.nsfw_content_detected):
    # Skip black images which are made when NSFW is detected.
    if not nsfw:
        num_generated_images += 1
        display(img)
        img.save(Path(save_dir).joinpath(f'{num_generated_images}.png'))

Explanation the some arguments above:
* `num_inference_steps`: how many steps to run the generation process for. ~50 is typical
* `guidance_scale`: tunes how much weight is given to the prompt during generation. 7.5 is the default, with larger numbers leading to stronger adherence to the prompt.
* `generator`: pass in a fixed `torch.Generator` instance for reproducible results.

`DetSDTextualInversionPipeline`'s `__call__` method accepts the same arguments as its underlying Huggingface `StableDiffusionPipeline` instance; see the [Hugging Face documentation](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline.__call__) for information on all available arguments.

## Step 4: Yes, this is a competition!

By running the cells above you have most likely gotten some nice Determined.AI logos with some watercolor painting theme. That's great, <b>but this is a competition!</b></br>
Repeat step 3.1 as many times as you want and try to adjust the following parameters:

- prompt (always include the `{first_concept}`)
- num_inference_steps=50
- guidance_scale=7.5

Your goal is to create the most amazing, creative, realistic, or astonishing image with the concept of a Determined.AI logo.
Make sure to save the images you like! Towards the end of the contest, move on to step 5, where your group will upload one image as your "best shot".
The image will be entered into the competition of this event and all attendees will vote on the most creative images. The winning group will receive a price!

<b>Good luck!</b>

## Step 5: Upload your image to participate in the voting

Once you are sure that you have the best possible image, please upload it on this page by clicking `Upload Photos`: https://app.kululu.me/album/gs5r92 </br></br>
<b>IMPORTANT:</b> When you selected your image, you have to option to click `Add Caption`. Please add all your names and the group number to the caption like so:

<img src="./detsd/example.png" alt="Example" width="450">

<b>NOTE:</b> Every group is only allowed to upload one image!